# RunnableWithMessageHistory

**RunnableWithMessageHistory**는 대화형 애플리케이션에서 메시지 기록을 관리하고, 이를 기반으로 상태를 유지하며 대화를 이어갈 수 있도록 돕는 클래스이다. 사용자와 AI 간의 대화에서 이전 대화의 맥락을 유지해야 할 때 유용하다.

## 주요 활용 사례

- **대화형 챗봇**: 이전 대화 내용을 기반으로 현재 질의에 적절히 응답해야 하는 경우에 사용된다.
- **단계적 처리**: 복잡한 데이터 처리 과정에서 여러 단계를 거쳐야 할 때, 현재 단계를 처리하기 위해 이전 단계의 결과를 참조해야 하는 경우에 활용된다.

## 특징

- 체인이 실행될 때마다 메시지 기록을 자동으로 업데이트하여, 개발자가 별도로 기록을 관리할 필요를 줄여준다.
- `session_id`를 기준으로 대화를 구분하며, 동일한 `session_id`를 사용하면 이전 대화의 맥락을 이어갈 수 있고, 새로운 `session_id`를 전달하면 새로운 대화로 인식한다.

## 작동 원리

- `Runnable` (Chain)과 `ChatMessageHistory`(메세지 저장소 객체)를 인자로 받아 생성한다.
- 체인이 실행되면서 생성되는 대화 메시지를 저장한다.

## Initializer 파라미터

- **runnable**: 체인(`Runnable`) 객체이다.
- **get_session_history**: `session_id`를 받아 해당 ID의 `ChatMessageHistory` 객체를 반환하는 함수이다. 이는 `BaseChatMessageHistory`를 상속받아 구현한 클래스의 인스턴스여야 한다.
- **input_messages_key**: 체인의 `invoke()` 메서드 호출 시 사용자 입력 메시지를 넣을 placeholder의 이름.
- **history_messages_key**: 채팅 기록에 저장된 메시지를 넣을 placeholder의 이름.

# 메시지 저장소 (Memory - ChatMessageHistory)

- **BaseChatMessageHistory**: 메시지 기록을 저장하고 관리하는 최상위 클래스. 저장 방식에 따라 이를 상속받아 다양한 클래스가 구현되어 있다.
- **InMemoryChatMessageHistory**: 메모리에 메시지를 저장하는 클래스이다. 빠르게 메시지를 관리할 수 있지만, 애플리케이션이 종료되면 데이터가 사라진다.
- 다양한 3rd party 저장소 연동 ChatMessageHistory 가 있다.
    - https://python.langchain.com/docs/integrations/memory/

이러한 구조를 통해 `RunnableWithMessageHistory`는 대화형 애플리케이션에서 효율적인 메시지 기록 관리와 상태 유지를 가능하게 한다. 

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

## ChatMessageHistory 실습

In [3]:
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage

m_history = InMemoryChatMessageHistory() # 대화내역을 메모리에 저장.
# 저장
m_history.add_message(("human", "안녕하세요."))  # Message객체
m_history.add_message(AIMessage("반갑습니다."))
m_history.add_message(HumanMessage("내 이름은 홍길동이야."))
m_history.add_message(SystemMessage("간결하게 답하세요."))

In [4]:
# 저장된 Message들 조회.
m_history.messages

[('human', '안녕하세요.'),
 AIMessage(content='반갑습니다.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='내 이름은 홍길동이야.', additional_kwargs={}, response_metadata={}),
 SystemMessage(content='간결하게 답하세요.', additional_kwargs={}, response_metadata={})]

In [6]:
# DB에 메세지들을 보관. => 영구적으로 보관: SQLChatMessageHistory
from langchain_community.chat_message_histories import SQLChatMessageHistory
from sqlalchemy import create_engine # DB 연결하는 함수.

engine = create_engine("sqlite:///chat_history.sqlite") # 저장할 Database와 연결. SQLite - "sqlite:///파일명"

s_history = SQLChatMessageHistory(
    session_id="test_id", 
    connection=engine
)
s_history.add_message(SystemMessage("간결하게 대답해 주세요."))
s_history.add_message(HumanMessage("안녕하세요."))

In [8]:
s_history.messages
s_history.get_messages()

[SystemMessage(content='간결하게 대답해 주세요.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='안녕하세요.', additional_kwargs={}, response_metadata={})]

## RunnableWithMessageHistory를 이용해 Chain 구성

In [ ]:
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory

from langchain_openai import ChatOpenAI

In [ ]:
store = {}  # key: session_id, 
            # value: InMemoryChatMessageHistory (session id별로 저장하는 기능이 없다.)
def get_session_history(session_id): # ChatMessageHistory 객체를 반환하는 함수.
    # store에서 session_id의 History객체를 찾아서 반환.
    # 없으면 생성해서 store 저장
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    
    return store[session_id]

history_msg = get_session_history("id-1")
# history_msg
store

{'id-1': InMemoryChatMessageHistory(messages=[])}

In [12]:
prompt_template = ChatPromptTemplate(
    [
        ("system", "질문에 대해 간결하게 대답해주세요. 정확하지 않으면 모른다고 대답하세요."),
        MessagesPlaceholder("history"), 
        ("human", "{query}")
    ]
)
model = ChatOpenAI(model="gpt-4o-mini")
runnable = prompt_template | model

# Chain + ChatMessageHistory => 대화 + 메세지 저장관리.
chain = RunnableWithMessageHistory(
    runnable=runnable, # chain 객체(RunnableSequence)
    get_session_history=get_session_history, # session_id의 ChatMessageHistory객체를 반환하는 함수.
    input_messages_key="query", # prompt template에 입력 내용을 넣을 변수명.
    history_messages_key="history" # prompt template에 대화내역을 넣으줄 변수명
)

In [14]:
result = chain.invoke(
    {"query":"Langchain에 장점 세가지를 설명해줘."}, # 입력데이터
    config={"configurable":{"session_id":"id-2"}}  # 설정 정보
)

In [16]:
print(result.content)

1. **모듈화**: Langchain은 다양한 구성 요소(모델, 데이터 소스, 체인 등)를 쉽게 조합할 수 있어 유연성과 재사용성을 높입니다.

2. **다양한 데이터 소스 통합**: Langchain은 여러 데이터 소스와 API를 통합할 수 있어, 복잡한 애플리케이션을 구축할 때 유용합니다.

3. **체인 구성**: 다양한 프롬프트 및 처리 단계를 체인 형태로 구성할 수 있어, 복잡한 작업을 효율적으로 처리할 수 있습니다.


In [17]:
store

{'id-1': InMemoryChatMessageHistory(messages=[]),
 'id-2': InMemoryChatMessageHistory(messages=[HumanMessage(content='Langchain에 장점 세가지를 설명해줘.', additional_kwargs={}, response_metadata={}), AIMessage(content='1. **모듈화**: Langchain은 다양한 구성 요소(모델, 데이터 소스, 체인 등)를 쉽게 조합할 수 있어 유연성과 재사용성을 높입니다.\n\n2. **다양한 데이터 소스 통합**: Langchain은 여러 데이터 소스와 API를 통합할 수 있어, 복잡한 애플리케이션을 구축할 때 유용합니다.\n\n3. **체인 구성**: 다양한 프롬프트 및 처리 단계를 체인 형태로 구성할 수 있어, 복잡한 작업을 효율적으로 처리할 수 있습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 126, 'prompt_tokens': 45, 'total_tokens': 171, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_bba3c8e70b', 'finish_reason': 'stop', 'logprobs': None}, id='run-64f94dc0-0d76-4421-9685-442960c1e598-0', usage_metadata={'input_to

In [18]:
result = chain.invoke(
    {"query":"세번째 장점에 대해 좀 더 자세하게 설명해줘."}, # 입력데이터
    config={"configurable":{"session_id":"id-2"}}  # 설정 정보
)

In [21]:
print(result.content)

Langchain의 체인 구성 기능은 여러 개의 처리 단계를 순차적으로 연결하여 복잡한 작업을 수행할 수 있도록 합니다. 사용자는 각 단계에서 특정 작업을 정의하고, 그 결과를 다음 단계의 입력으로 사용할 수 있습니다. 

예를 들어, 데이터 수집, 전처리, 모델 예측, 후처리 등의 과정을 체인으로 구성할 수 있습니다. 이렇게 하면 각 단계의 결과가 자동으로 다음 단계에 전달되며, 전체 프로세스를 간소화하고 명확하게 관리할 수 있습니다. 

또한, 체인은 쉽게 수정하거나 확장할 수 있어, 새로운 기능이나 알고리즘을 추가할 때 유용합니다. 이를 통해 개발자는 복잡한 애플리케이션을 보다 효율적으로 구축하고 유지관리할 수 있습니다.


# RunnableWithMessageHistory + Memory class

- Memory class로 저장 방식 설정
- BaseChatMessageHistory로 어디에 저장할 지 설정
- RunnableWithMessageHistory로 chain 설정

In [22]:
from langchain.memory import ConversationBufferWindowMemory

In [23]:
store

{'id-1': InMemoryChatMessageHistory(messages=[]),
 'id-2': InMemoryChatMessageHistory(messages=[HumanMessage(content='Langchain에 장점 세가지를 설명해줘.', additional_kwargs={}, response_metadata={}), AIMessage(content='1. **모듈화**: Langchain은 다양한 구성 요소(모델, 데이터 소스, 체인 등)를 쉽게 조합할 수 있어 유연성과 재사용성을 높입니다.\n\n2. **다양한 데이터 소스 통합**: Langchain은 여러 데이터 소스와 API를 통합할 수 있어, 복잡한 애플리케이션을 구축할 때 유용합니다.\n\n3. **체인 구성**: 다양한 프롬프트 및 처리 단계를 체인 형태로 구성할 수 있어, 복잡한 작업을 효율적으로 처리할 수 있습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 126, 'prompt_tokens': 45, 'total_tokens': 171, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_bba3c8e70b', 'finish_reason': 'stop', 'logprobs': None}, id='run-64f94dc0-0d76-4421-9685-442960c1e598-0', usage_metadata={'input_to

In [50]:
store = {}
config = {"configurable": {"session_id":"chat_message_1"}}

def get_session_history(session_id):
    
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
        return store[session_id]
    
    memory = ConversationBufferWindowMemory(
        chat_memory=store[session_id], # 메세지 저장소 추가.
        k=2,
        return_messages=True,
        message_key="history"
    )
    # 1. memory에서 저장된 message들을 조회
    message_list = memory.load_memory_variables({})['history']# list[Message]
    # 2. 조회한 message들을 InMemoryChatMessageHistory에 추가
    store[session_id] = InMemoryChatMessageHistory(messages=message_list)
    # 3. 2의 InMemoryChatMessageHistory를 반환.
    return store[session_id]

In [51]:
prompt_template = ChatPromptTemplate(
    [
        ("system", "질문에 대해 간결하게 대답해주세요. 정확하지 않으면 모른다고 대답하세요."),
        MessagesPlaceholder("history"), 
        ("human", "{query}")
    ]
)
model = ChatOpenAI(model="gpt-4o-mini")
runnable = prompt_template | model

# Chain + ChatMessageHistory => 대화 + 메세지 저장관리.
chain = RunnableWithMessageHistory(
    runnable=runnable, # chain 객체(RunnableSequence)
    get_session_history=get_session_history, # session_id의 ChatMessageHistory객체를 반환하는 함수.
    input_messages_key="query", # prompt template에 입력 내용을 넣을 변수명.
    history_messages_key="history" # prompt template에 대화내역을 넣으줄 변수명
)

In [52]:
result = chain.invoke({"query":"내 이름은 홍길동입니다. 나는 20세 입니다."}, config)

In [53]:
print(result)

content='반갑습니다, 홍길동님! 20세라니 젊으시군요. 도움이 필요하시면 말씀해 주세요.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 47, 'total_tokens': 76, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_818c284075', 'finish_reason': 'stop', 'logprobs': None} id='run-00d303a9-5054-4576-8e70-d194f2439dc5-0' usage_metadata={'input_tokens': 47, 'output_tokens': 29, 'total_tokens': 76, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [54]:
store

{'chat_message_1': InMemoryChatMessageHistory(messages=[HumanMessage(content='내 이름은 홍길동입니다. 나는 20세 입니다.', additional_kwargs={}, response_metadata={}), AIMessage(content='반갑습니다, 홍길동님! 20세라니 젊으시군요. 도움이 필요하시면 말씀해 주세요.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 47, 'total_tokens': 76, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_818c284075', 'finish_reason': 'stop', 'logprobs': None}, id='run-00d303a9-5054-4576-8e70-d194f2439dc5-0', usage_metadata={'input_tokens': 47, 'output_tokens': 29, 'total_tokens': 76, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})])}

In [55]:
result = chain.invoke({"query":"내가 몇살인가요?"}, config)

In [56]:
result

AIMessage(content='20세라고 하셨습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 91, 'total_tokens': 98, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_bba3c8e70b', 'finish_reason': 'stop', 'logprobs': None}, id='run-26b48777-ff7d-40cb-8dc0-db2ddfd4495e-0', usage_metadata={'input_tokens': 91, 'output_tokens': 7, 'total_tokens': 98, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [44]:
store

{'chat_message_1': InMemoryChatMessageHistory(messages=[HumanMessage(content='내 이름은 홍길동입니다. 나는 20세 입니다.', additional_kwargs={}, response_metadata={}), AIMessage(content='안녕하세요, 홍길동님! 만나서 반갑습니다. 20세시군요. 무엇을 도와드릴까요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 47, 'total_tokens': 76, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_bba3c8e70b', 'finish_reason': 'stop', 'logprobs': None}, id='run-daf9573d-a895-48f1-944c-6973f3178c2e-0', usage_metadata={'input_tokens': 47, 'output_tokens': 29, 'total_tokens': 76, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}), HumanMessage(content='내가 몇살인가요?', additional_kwargs={}, response_metadata={}), AI

In [57]:
result = chain.invoke({"query":"내 이름은 무엇인가요?"}, config)

In [58]:
print(result.content)

당신의 이름은 홍길동입니다.


In [47]:
store

{'chat_message_1': InMemoryChatMessageHistory(messages=[HumanMessage(content='내 이름은 홍길동입니다. 나는 20세 입니다.', additional_kwargs={}, response_metadata={}), AIMessage(content='안녕하세요, 홍길동님! 만나서 반갑습니다. 20세시군요. 무엇을 도와드릴까요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 47, 'total_tokens': 76, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_bba3c8e70b', 'finish_reason': 'stop', 'logprobs': None}, id='run-daf9573d-a895-48f1-944c-6973f3178c2e-0', usage_metadata={'input_tokens': 47, 'output_tokens': 29, 'total_tokens': 76, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}), HumanMessage(content='내가 몇살인가요?', additional_kwargs={}, response_metadata={}), AI

In [59]:
result = chain.invoke({"query":"내 이름과 나이를 다시 알려주세요?"}, config)

In [60]:
result

AIMessage(content='죄송하지만, 당신의 이름과 나이를 알고 있지 않습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 89, 'total_tokens': 104, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_bba3c8e70b', 'finish_reason': 'stop', 'logprobs': None}, id='run-d11a804b-e91f-492e-be10-8752d8209a99-0', usage_metadata={'input_tokens': 89, 'output_tokens': 15, 'total_tokens': 104, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [63]:
from pprint import pprint
pprint(store['chat_message_1'].messages)

[HumanMessage(content='내가 몇살인가요?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='20세라고 하셨습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 91, 'total_tokens': 98, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_bba3c8e70b', 'finish_reason': 'stop', 'logprobs': None}, id='run-26b48777-ff7d-40cb-8dc0-db2ddfd4495e-0', usage_metadata={'input_tokens': 91, 'output_tokens': 7, 'total_tokens': 98, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}),
 HumanMessage(content='내 이름은 무엇인가요?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='당신의 이름은 홍길동입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_